# Using a GNN model for user cell prediction and making predictions using Inductive inference

This notebook goes over how to use the Neptune ML sagemaker endpoint to make predictions with Gremlin Queries on new nodes

This notebook relies on having run notebook 3a-TransductiveMode-CellPrediction to get a Neptune ML SageMaker model endpoint.


In [ ]:
# fill the inductive endpoint with the same endpoint from the previous notebook
endpoint_inductive = '<replace with endpoint>'

## Inductive prediction with Gremlin queries
### load new user and some user edges

In [ ]:
%load -s s3://neptuneml-data/telco-graph/inductive-data -f csv -p OVERSUBSCRIBE

In [ ]:
%%gremlin
g.V()
.hasId("user_20000")

In [ ]:
%%gremlin
g.V()
.hasId("user_20000")
.outE()
.hasLabel("user_live_cell")
.inV()
.valueMap(true, "name")
.groupCount()
.unfold()
.order()
.by(values, desc)

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint_inductive}").
    with("Neptune#ml.limit",10).
      V().hasId('user_20000').
        out('user_live_cell')
        .with("Neptune#ml.prediction")
        .with("Neptune#ml.inductiveInference")
        .hasLabel('cell')
        .id()

<div style="background-color:#eeeeee; padding:20px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Information</b>: both cell_34832, cell_19832 are in the ground truth linked to user_20000 </div>
